# paintings_db

In [1]:
!pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import yaml

In [3]:
with open("my_secret_file.yaml", "r") as stream:
    try:
        data = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [4]:
import mysql.connector

In [5]:
con = mysql.connector.connect(host="localhost", port=3306, user=data['username'], password=data['password'])

In [6]:
cur = con.cursor()

## Create

In [70]:
cur.execute("CREATE SCHEMA IF NOT EXISTS paintings_schema")

In [71]:
cur.execute("""CREATE TABLE paintings_schema.painting (
  id_painting INT NOT NULL AUTO_INCREMENT,
  title VARCHAR(120) NULL,
  creation_date VARCHAR(30) NULL,
  id_artist VARCHAR(60) NULL,
  id_museum VARCHAR(60) NULL,
  id_medium VARCHAR(60) NULL,
  id_art_movement VARCHAR(45) NULL,
  PRIMARY KEY (id_painting))
  """)

In [72]:
import pandas as pd

In [73]:
data_paintings = pd.read_csv('paintings_db - painting.csv')   
df_paintings = pd.DataFrame(data_paintings)

In [74]:
df_paintings.head()

,title,creation_date,id_artist,id_museum,id_medium,id_art_movement
0,Almond Blossoms,1890,Vincent van Gogh,Van Gogh Museum,oil on canvas,Post Impressionism
1,American Gothic,1930,Grant Wood,Art Institute of Chicago,oil on beaver board,Regionalism
2,Archer,1919,Theo van Doesburg,Museum of Fine Arts,pencil and gouache on paper,Dada
3,Au Moulin Rouge,1892-1895,Henri de Toulouse-Lautrec,Art Institute of Chicago,oil on canvas,Post Impressionism
4,Beata Beatrix,1864,Dante Gabriel Rossetti,Tate Britain,oil on canvas,Aestheticism


In [75]:
query = "INSERT INTO paintings_schema.painting (title, creation_date, id_artist, id_museum, id_medium, id_art_movement) VALUES (%s, %s, %s, %s, %s, %s)"
for row in df_paintings.itertuples():
    data = (row.title, row.creation_date, row.id_artist, row.id_museum, row.id_medium, row.id_art_movement)
    cur.execute(query, data)
con.commit()

## Read

In [76]:
cur.execute("SELECT * FROM paintings_schema.painting")

In [77]:
res = cur.fetchall()

In [78]:
len(res)

74

In [79]:
res[:2]

[(1,
  'Almond Blossoms',
  '1890',
  'Vincent van Gogh',
  'Van Gogh Museum',
  'oil on canvas',
  'Post Impressionism'),
 (2,
  'American Gothic',
  '1930',
  'Grant Wood',
  'Art Institute of Chicago',
  'oil on beaver board',
  'Regionalism')]

## Update

In [80]:
cur.execute("UPDATE paintings_schema.painting SET creation_date = NULL WHERE (creation_date = '-')")
cur.execute("UPDATE paintings_schema.painting SET id_medium = NULL WHERE (id_medium = '-')")
con.commit()

## artists

In [81]:
data_artists = pd.read_csv('paintings_db - artist.csv')   
df_artists = pd.DataFrame(data_artists)

In [82]:
df_artists.head()

,name,born,died,origin
0,Albrecht Dürer,1471,1528,German
1,Alice Bailly,1872,1938,Swiss
2,Andy Warhol,1928,1987,American
3,Banksy,-,-,-
4,Carl Andre,1935,-,American


In [83]:
cur.execute("""CREATE TABLE paintings_schema.artist (
  id_artist INT NOT NULL AUTO_INCREMENT,
  name VARCHAR(120) NULL,
  born VARCHAR(30) NULL,
  died VARCHAR(30) NULL,
  origin VARCHAR(60) NULL,
  PRIMARY KEY (id_artist))
  """)

In [84]:
query = "INSERT INTO paintings_schema.artist (name, born, died, origin) VALUES (%s, %s, %s, %s)"
for row in df_artists.itertuples():
    data = (row.name, row.born, row.died, row.origin)
    cur.execute(query, data)
con.commit()

In [85]:
cur.execute("UPDATE paintings_schema.artist SET born = NULL WHERE (born = '-')")
cur.execute("UPDATE paintings_schema.artist SET died = NULL WHERE (died = '-')")
cur.execute("UPDATE paintings_schema.artist SET origin = NULL WHERE (origin = '-')")
con.commit()

In [86]:
cur.execute(
    """
    UPDATE paintings_schema.painting
    INNER JOIN paintings_schema.artist
    ON paintings_schema.painting.id_artist = paintings_schema.artist.name
    SET paintings_schema.painting.id_artist = paintings_schema.artist.id_artist
    WHERE paintings_schema.painting.id_artist = paintings_schema.artist.name
    """
    )
con.commit()

In [88]:
cur.execute(
    """
    ALTER TABLE paintings_schema.painting
    MODIFY id_artist INT,
    ADD CONSTRAINT fk_id_artist
    FOREIGN KEY (id_artist) REFERENCES paintings_schema.artist(id_artist)
    """
    )
con.commit()

## museums

In [89]:
data_museums = pd.read_csv('paintings_db - museum.csv')   
df_museums = pd.DataFrame(data_museums)

In [90]:
df_museums.head()

,name,id_location
0,Albertina Museum,"Vienna, Austria"
1,Albright Art Gallery,"New York, USA"
2,Albright-Knox Art Gallery,"Buffalo, USA"
3,Alex Reid and Lefevre,"London, England"
4,Art Gallery of New South Wales,"Sydney, Australia"


In [91]:
cur.execute("""CREATE TABLE paintings_schema.museum (
  id_museum INT NOT NULL AUTO_INCREMENT,
  name VARCHAR(120) NULL,
  id_location VARCHAR(60) NULL,
  PRIMARY KEY (id_museum))
  """)

In [92]:
query = "INSERT INTO paintings_schema.museum (name, id_location) VALUES (%s, %s)"
for row in df_museums.itertuples():
    data = (row.name, row.id_location)
    cur.execute(query, data)
con.commit()

In [93]:
cur.execute(
    """
    UPDATE paintings_schema.painting
    INNER JOIN paintings_schema.museum
    ON paintings_schema.painting.id_museum = paintings_schema.museum.name
    SET paintings_schema.painting.id_museum = paintings_schema.museum.id_museum
    WHERE paintings_schema.painting.id_museum = paintings_schema.museum.name
    """
    )
con.commit()

In [94]:
cur.execute(
    """
    ALTER TABLE paintings_schema.painting
    MODIFY id_museum INT,
    ADD CONSTRAINT fk_id_museum
    FOREIGN KEY (id_museum) REFERENCES paintings_schema.museum(id_museum)
    """
    )
con.commit()

## mediums

In [95]:
cur.execute("""CREATE TABLE paintings_schema.medium (
  id_medium INT NOT NULL AUTO_INCREMENT,
  title VARCHAR(60) NULL,
  PRIMARY KEY (id_medium))
  """)

In [96]:
cur.execute("""INSERT INTO paintings_schema.medium (title)
    SELECT DISTINCT id_medium
    FROM paintings_schema.painting
    WHERE id_medium IS NOT NULL
    """)
con.commit()

In [97]:
cur.execute(
    """
    UPDATE paintings_schema.painting
    INNER JOIN paintings_schema.medium
    ON paintings_schema.painting.id_medium = paintings_schema.medium.title
    SET paintings_schema.painting.id_medium = paintings_schema.medium.id_medium
    WHERE paintings_schema.painting.id_medium = paintings_schema.medium.title
    """
    )
con.commit()

In [98]:
cur.execute(
    """
    ALTER TABLE paintings_schema.painting
    MODIFY id_medium INT,
    ADD CONSTRAINT fk_id_medium
    FOREIGN KEY (id_medium) REFERENCES paintings_schema.medium(id_medium)
    """
    )
con.commit()

## art movements

In [99]:
data_art_movements = pd.read_csv('paintings_db - art_movement.csv')   
df_art_movements = pd.DataFrame(data_art_movements)

In [100]:
df_art_movements.head()

,title,dates
0,Abstract Expressionism,1946-...
1,Academic Art,1560-1900
2,Aestheticism,1868-1900
3,Art Deco,1920-1939
4,Art Nouveau,1890-1914


In [101]:
cur.execute("""CREATE TABLE paintings_schema.art_movement (
  id_art_movement INT NOT NULL AUTO_INCREMENT,
  title VARCHAR(120) NULL,
  dates VARCHAR(30) NULL,
  PRIMARY KEY (id_art_movement))
  """)

In [102]:
query = "INSERT INTO paintings_schema.art_movement (title, dates) VALUES (%s, %s)"
for row in df_art_movements.itertuples():
    data = (row.title, row.dates)
    cur.execute(query, data)
con.commit()

In [103]:
cur.execute(
    """
    UPDATE paintings_schema.painting
    INNER JOIN paintings_schema.art_movement
    ON paintings_schema.painting.id_art_movement = paintings_schema.art_movement.title
    SET paintings_schema.painting.id_art_movement = paintings_schema.art_movement.id_art_movement
    WHERE paintings_schema.painting.id_art_movement = paintings_schema.art_movement.title
    """
    )
con.commit()

In [104]:
cur.execute(
    """
    ALTER TABLE paintings_schema.painting
    MODIFY id_art_movement INT,
    ADD CONSTRAINT fk_id_art_movement
    FOREIGN KEY (id_art_movement) REFERENCES paintings_schema.art_movement(id_art_movement)
    """
    )
con.commit()

## exhibitions

In [105]:
data_exhibitions = pd.read_csv('paintings_db - exhibition.csv')   
df_exhibitions = pd.DataFrame(data_exhibitions)

In [106]:
df_exhibitions.head()

,id_painting,title,dates,id_museum
0,Almond Blossoms,Bloesem,21 March 2011 - 21 June 2011,Van Gogh Museum
1,Almond Blossoms,Eeuwfeest Vincent van Gogh,24 May 1953 - 19 July 1953,Kröller-Müller Museum
2,Almond Blossoms,Eeuwfeest Vincent van Gogh,23 July 1953 - 20 September 1953,Stedelijk Museum Amsterdam
3,Almond Blossoms,Japan : Van Gogh's Utopia,7 June 1991 - 22 September 1991,Rijksmuseum Vincent van Gogh
4,Almond Blossoms,L'art contemporain. Salon 1914 / Kunst van Hed...,7 March 1914 - 5 April 1914,Zaal Comité voor Artistieke Werking


In [107]:
cur.execute("""CREATE TABLE paintings_schema.exhibition (
  id_exhibition INT NOT NULL AUTO_INCREMENT,
  id_painting VARCHAR(120) NULL,
  title VARCHAR(256) NULL,
  dates VARCHAR(45) NULL,
  id_museum VARCHAR(120) NULL,
  PRIMARY KEY (id_exhibition))
  """)

In [108]:
query = "INSERT INTO paintings_schema.exhibition (id_painting, title, dates, id_museum) VALUES (%s, %s, %s, %s)"
for row in df_exhibitions.itertuples():
    data = (row.id_painting, row.title, row.dates, row.id_museum)
    cur.execute(query, data)
con.commit()

In [109]:
cur.execute("UPDATE paintings_schema.exhibition SET title = NULL WHERE (title = '-')")
cur.execute("UPDATE paintings_schema.exhibition SET dates = NULL WHERE (dates = '-')")
cur.execute("UPDATE paintings_schema.exhibition SET id_museum = NULL WHERE (id_museum = '-')")
con.commit()

In [110]:
cur.execute(
    """
    UPDATE paintings_schema.exhibition
    INNER JOIN paintings_schema.painting
    ON paintings_schema.exhibition.id_painting = paintings_schema.painting.title
    SET paintings_schema.exhibition.id_painting = paintings_schema.painting.id_painting
    WHERE paintings_schema.exhibition.id_painting = paintings_schema.painting.title
    """
    )
con.commit()

In [111]:
cur.execute(
    """
    ALTER TABLE paintings_schema.exhibition
    MODIFY id_painting INT,
    ADD CONSTRAINT fk_id_painting
    FOREIGN KEY (id_painting) REFERENCES paintings_schema.painting(id_painting)
    """
    )
con.commit()

In [112]:
cur.execute(
    """
    UPDATE paintings_schema.exhibition
    INNER JOIN paintings_schema.museum
    ON paintings_schema.exhibition.id_museum = paintings_schema.museum.name
    SET paintings_schema.exhibition.id_museum = paintings_schema.museum.id_museum
    WHERE paintings_schema.exhibition.id_museum = paintings_schema.museum.name
    """
    )
con.commit()

In [114]:
cur.execute(
    """
    ALTER TABLE paintings_schema.exhibition
    MODIFY id_museum INT,
    ADD CONSTRAINT fk_exhibition_id_museum
    FOREIGN KEY (id_museum) REFERENCES paintings_schema.museum(id_museum)
    """
    )
con.commit()

## locations

In [115]:
cur.execute("""CREATE TABLE paintings_schema.location (
  id_location INT NOT NULL AUTO_INCREMENT,
  locality VARCHAR(30) NULL,
  country VARCHAR(30) NULL,
  PRIMARY KEY (id_location))
  """)

In [116]:
cur.execute(
    """
    USE paintings_schema
    """)

In [117]:
cur.execute(
    """
    CREATE FUNCTION SPLIT_STR(
      x VARCHAR(255),
      delim VARCHAR(12),
      pos INT
        )
    RETURNS VARCHAR(255) DETERMINISTIC
    BEGIN 
        RETURN REPLACE(SUBSTRING(SUBSTRING_INDEX(x, delim, pos),
        LENGTH(SUBSTRING_INDEX(x, delim, pos -1)) + 1),
        delim, '');
    END
    """)

In [118]:
cur.execute(
    """
    INSERT INTO paintings_schema.location (locality, country)
    SELECT DISTINCT SPLIT_STR(id_location, ', ', 1), SPLIT_STR(id_location, ', ', 2)
    FROM paintings_schema.museum
    """)

In [119]:
cur.execute(
    """
    UPDATE paintings_schema.museum
    INNER JOIN paintings_schema.location
    SET paintings_schema.museum.id_location = paintings_schema.location.id_location
    WHERE paintings_schema.museum.id_location LIKE CONCAT('%', paintings_schema.location.locality, '%')
    """
    )
con.commit()

In [120]:
cur.execute(
    """
    ALTER TABLE paintings_schema.museum
    MODIFY id_location INT,
    ADD CONSTRAINT fk_id_location
    FOREIGN KEY (id_location) REFERENCES paintings_schema.location(id_location)
    """
    )
con.commit()

In [121]:
cur.execute("UPDATE paintings_schema.location SET locality = NULL WHERE (locality = '-')")
con.commit()

## Delete

In [ ]:
cur.execute("DROP DATABASE paintings_schema")